In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GEMINI_API_KEY"]=os.getenv("GEMINI_API_KEY")


In [2]:
from langchain_community.document_loaders import PythonLoader
# % pip install pythonloader
loader = PythonLoader("app.py")
pythondocs = loader.load()


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(pythondocs)
documents[:5]

[Document(metadata={'source': 'app.py'}, page_content='from fastapi import FastAPI\nfrom langchain.prompts import ChatPromptTemplate\nfrom langchain_ollama import ChatOllama\nfrom langchain_google_genai import ChatGoogleGenerativeAI\nfrom langchain_community.llms import Ollama\nfrom langserve import add_routes\nimport uvicorn\nimport os\nfrom dotenv import load_dotenv\n\n# Load environment variables\nload_dotenv()\n\napp = FastAPI(\n    title="Simple Chatbot API",\n    description="A simple API for interacting with the TinyLlama chatbot using LangChain and Ollama.",\n    version="1.0.0"\n)\n\n\n\n\nllm1 = ChatOllama(model="tinyllama")\ngemini = ChatGoogleGenerativeAI(\n    model="models/gemini-1.5-flash",\n    google_api_key=os.getenv("GEMINI_API_KEY")\n)\n\n\n\nprompt1 = ChatPromptTemplate.from_template(\n    "You are a helpful assitant prompt : {topic}."\n)\nprompt2 = ChatPromptTemplate.from_template(\n    "You are a helpful assistant prompt : {topic}."\n)\n\n\n\nadd_routes(\n    app

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
import os

df = FAISS.from_documents(
    documents[:20], 
    GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        google_api_key=os.environ["GEMINI_API_KEY"]
    )
)
retriever = df.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000207FECFB250>, search_kwargs={})

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents, GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=os.environ["GEMINI_API_KEY"]))




In [14]:
# AI-powered Error Detection and Code Fixing
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
    google_api_key=os.environ["GEMINI_API_KEY"]
)

# Create a prompt template for error detection and fixing
error_fixing_prompt = ChatPromptTemplate.from_template("""
You are an expert Python code reviewer and debugger. Analyze the following code for errors and issues:

CODE TO ANALYZE:
{code}

Please:
1. Identify all errors, bugs, and potential issues
2. Explain what each error is and why it's problematic
3. Provide the corrected version of the code
4. Add comments explaining the fixes

Format your response as:
## ERRORS FOUND:
[List all errors with explanations]

## CORRECTED CODE:
```python
[Provide the fixed code with comments]
```

## EXPLANATIONS:
[Explain the fixes made]
""")

# Get the full code content
full_code = "\n".join([doc.page_content for doc in documents])

print("🔍 Analyzing code for errors...")
print("=" * 50)

# Create the chain and invoke it
chain = error_fixing_prompt | llm
result = chain.invoke({"code": full_code})

print(result.content)

🔍 Analyzing code for errors...
## ERRORS FOUND:

1. **Duplicate Route:** The code registers the `/ollama` route twice using `add_routes`. This will lead to a conflict and only one of the routes will be functional, potentially leading to unexpected behavior.

2. **Inconsistent Prompt Templates:** While both `prompt1` and `prompt2` are almost identical (only a minor difference in "assitant" vs "assistant"), using two separate prompts is redundant.  It's better to use a single prompt template.

3. **Missing Error Handling:** The code lacks error handling.  If `os.getenv("GEMINI_API_KEY")` returns `None` (meaning the environment variable isn't set),  `ChatGoogleGenerativeAI` will raise an exception, crashing the application.  Similarly, network issues with Ollama or Google Gemini could also cause crashes.

4. **Potential for Large Response Sizes:**  The code doesn't handle potential large responses from the LLMs which could lead to memory issues or slowdowns.  Streaming responses would be 

In [ ]:
# Interactive Code Error Checker
def check_code_errors():
    print("📝 Interactive Code Error Checker")
    print("Enter your code (press Enter twice to finish):")
    print("-" * 40)
    
    lines = []
    while True:
        line = input()
        if line == "" and len(lines) > 0 and lines[-1] == "":
            break
        lines.append(line)
    
    # Remove the last empty line
    if lines and lines[-1] == "":
        lines.pop()
    
    user_code = "\n".join(lines)
    
    if not user_code.strip():
        print("❌ No code provided!")
        return
    
    print("\n🔍 Analyzing your code...")
    print("=" * 50)
    
    # Use the same error-fixing chain
    result = chain.invoke({"code": user_code})
    print(result.content)

# Call the interactive checker
check_code_errors()